# Sales Copilot

Additional concepts:

- Copilot determines intent. Either goes to load balance endpoint or Assistant depending on user intent.
- Thread management for multiple users
  - Using shelves to store the user threads
  - Thread will be created or reused as different users access the assistant
- Relationships in files

## Diagram

- **Note**: Not implemented, promptflow and weather services

![Architecture Diagram](../images/copilot-with-assistant.png)

### Load the required libraries

In [1]:
import os
import sales_agent as agent

from AgentSettings import AgentSettings
from openai import AzureOpenAI

### Load the environment variables

In [2]:
settings = AgentSettings()

### Create the AzureOpenAI clients

In [3]:
load_balanced_client = AzureOpenAI(api_key=settings.api_key, api_version=settings.api_version, azure_endpoint=settings.api_endpoint)

In [4]:
sales = agent.get_agent(settings)

data/sales/sellers.csv
data/sales/customers.csv
data/sales/orders.csv
data/sales/inventory.csv


### Determine intent semantically

In [5]:
def semantic_intent(prompt:str)->str:
    prompt_template = """system:
You are an agent that can determine intent from the following list of intents and return the intent that best matches the user's question.
 
List of intents:
SalesAgent: questions or operations related to calculations, customers, sales, orders, inventory, items, and sellers.
OtherAgent: any other question

user:
<QUESTION>

Output in ONE word."""


    full_prompt = prompt_template.replace("<QUESTION>", prompt)
    completion = load_balanced_client.chat.completions.create(
            model=settings.model_deployment,
            messages=[
                {
                    "role": "user",
                    "content": full_prompt,
                },
            ],
            max_tokens=2,
            temperature=0.1           
        )    
    try:
        return completion.choices[0].message.content
    except:
        return "Unknown"
    

### Process for intent

In [6]:
def process_for_intent(user: str, user_id: str, prompt: str):
    intent = semantic_intent(prompt)
    print(f"Intent: {intent}")
    
    if intent=="SalesAgent":
        sales.process_prompt(user, user_id, "What customers are in Florida?")
    elif intent=="OtherAgent":
        completion = load_balanced_client.chat.completions.create(
            model=settings.model_deployment,
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ]
        )
        print(completion.choices[0].message.content)
        
    else:        
        print("Unknown Intent")

### Have a conversation with the Assistant

In [7]:
process_for_intent("Suzan", "user_123", "What is the speed of light?")

Intent: OtherAgent
The speed of light in a vacuum is a fundamental physical constant and is exactly 299,792,458 meters per second (approximately 186,282 miles per second). This constant is often denoted by the letter 'c' and is crucial in the field of physics, particularly in the theory of relativity formulated by Albert Einstein.


In [8]:
process_for_intent("John", "user_123", "What customers are in Florida?")

Intent: SalesAgent
processing ...
user: John:
What customers are in Florida?

assistant:
To determine which customers are in Florida, I will need to examine the contents of the files you've uploaded to see if any of them contains customer information with location data. I'll start by checking the first file you uploaded. Please give me a moment to process the file and find the information for you.

assistant:
The first file contains customer information, including the states they are from. Let's check all the files and then filter the data to find customers located in Florida (FL). Please bear with me as I process each file.

assistant:
Based on the files provided, I found customer information with associated states in the second file. Here is the list of customers in Florida (FL):

- John Doe (CustomerID: 1000)

Would you like to know anything else, Jane Doe?

Deleted thread:  thread_IF0m2yAewpyb9iipSgQS99n4


In [ ]:
process_for_intent("Mary", "user_234", "What seller has had the most sales?")

In [ ]:
process_for_intent("John", "user_123", "What is the most sold product?")

In [ ]:
process_for_intent("Mary", "user_234", "Chart product sales by State.")

### Cleanup

In [9]:
sales.cleanup()

AssistantDeleted(id='asst_AnzFXedFYPQGBQcsjI2LinRr', deleted=True, object='assistant.deleted')
Deleting:  0  threads.
Deleting:  4  files.
FileDeleted(id='assistant-oifyC5pCw8X3X6SZyiPdYpVd', deleted=True, object='file')
FileDeleted(id='assistant-cWJ1W2fqNjPezJX0S9tTXPXp', deleted=True, object='file')
FileDeleted(id='assistant-8UaBazJ7op3OkBnlOTYh8Sxy', deleted=True, object='file')
FileDeleted(id='assistant-qDyxKd3oLsOzvTI22IExUKvo', deleted=True, object='file')
